In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
import numpy as np
import pandas as pd
import os
from keras import layers
import keras
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D,MaxPool2D ,UpSampling2D, Flatten, Input,BatchNormalization
# from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam, Adadelta, Adagrad
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from math import log10,sqrt


In [4]:
import imageio
import glob
import os
import time
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import layers
from IPython import display
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from tensorflow import keras

In [5]:
train_images=sorted(os.listdir('/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'))
test_images=sorted(os.listdir('/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'))

train_images=train_images[:1500]
test_images=test_images+train_images[5000:]

In [7]:
train_image,test_image=[],[]
for im in train_images:
    try:
        img=image.load_img('/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'+im,target_size=(256,256))
    except:
        print("Couldnot Load Image")
    img=image.img_to_array(img)
    img=img/255
    train_image.append(img)
    
train_df=np.array(train_image)

In [ ]:
def plot_img(dataset):
  f,ax=plt.subplots(1,5)
  f.set_size_inches(40,20)
  for i in range(5,10):
    ax[i-5].imshow(dataset[i], cmap='gray')
  plt.show()

In [8]:
def add_noise(image):
  row,col,ch=image.shape
  mean=0
  sigma=1
  gauss=np.random.normal(mean,sigma,(row,col,ch))
  poisson=np.random.poisson(image)
  gauss=gauss.reshape(row,col,ch)
  noisy=image+gauss*0.5+poisson
  return noisy
     

In [10]:
noised_train = []
noised_test = []
for img in train_df:
    noised_train.append(add_noise(img))
    noised_test.append(add_noise(img))

In [12]:
def encoder(input_encoder):
    inputs = keras.Input(shape=input_encoder, name='input_layer')
    # Block 1
    x = layers.Conv2D(32, kernel_size=3, strides= 1, padding='same', name='conv_1')(inputs)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.LeakyReLU(name='lrelu_1')(x)

    # Block 2
    x = layers.Conv2D(64, kernel_size=3, strides= 2, padding='same', name='conv_2')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.LeakyReLU(name='lrelu_2')(x)
    
    # Block 3
    x = layers.Conv2D(64, 3, 2, padding='same', name='conv_3')(x)
    x = layers.BatchNormalization(name='bn_3')(x)
    x = layers.LeakyReLU(name='lrelu_3')(x)
     
    # Block 4
    x = layers.Conv2D(64, 3, 1, padding='same', name='conv_4')(x)
    x = layers.BatchNormalization(name='bn_4')(x)
    x = layers.LeakyReLU(name='lrelu_4')(x)
     
    # Final Block
    flatten = layers.Flatten()(x)
    bottleneck = layers.Dense(200, name='dense_1')(flatten)
    model = tf.keras.Model(inputs, bottleneck, name="Encoder")
    return model

In [13]:
def decoder(input_decoder):
    # Initial Block
    inputs = keras.Input(shape=input_decoder, name='input_layer')
    x = layers.Dense(262144, name='dense_1')(inputs)
    #print("After Dense:",x.shape)
    x = tf.reshape(x, [-1, 64, 64, 64], name='Reshape_Layer')
    #print("After Reshape: ",x.shape) 
    # Block 1
    x = layers.Conv2DTranspose(64, 3, strides= 1, padding='same',name='conv_transpose_1')(x)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.LeakyReLU(name='lrelu_1')(x)
    #print("First Transpose:",x.shape) 
    # Block 2
    x = layers.Conv2DTranspose(64, 3, strides= 2, padding='same', name='conv_transpose_2')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.LeakyReLU(name='lrelu_2')(x)
    #print("Second Transpose:",x.shape)
    # Block 3
    x = layers.Conv2DTranspose(32, 3, 2, padding='same', name='conv_transpose_3')(x)
    x = layers.BatchNormalization(name='bn_3')(x)
    x = layers.LeakyReLU(name='lrelu_3')(x)
    #print("Third Transpose:",x.shape)
    # Block 4
    outputs = layers.Conv2DTranspose(3, 3, 1,padding='same', activation='sigmoid', name='conv_transpose_4')(x)
    #print("Outputs:",outputs.shape)
    model = tf.keras.Model(inputs, outputs, name="Decoder")
    return model

In [14]:
enc=encoder((256,256,3))
dec=decoder((200))

### I have noticed here that the validation loss increases from what it's previous value was which clearly is a sign of OverFitting.
### Also I guess, it stores the Model's weights in memory and this surprinsingly being a model with many weights overloads and causes memory issues

In [15]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_df).batch(50)
noised_dataset=tf.data.Dataset.from_tensor_slices(noised_train).batch(50)

In [16]:
optimizer = tf.keras.optimizers.legacy.Adam(lr = 0.0005)
loss=0
#Loss Function
def ae_loss(y_true, y_pred):
    global loss
    loss=loss
    try:
        loss=  tf.reduce_mean(tf.square(tf.subtract(y_pred, y_true)))
    except:
        print("Error encountered in Loss computation")
        print("Shape of y_pred:",y_pred.shape)
        print("Shape of y_true:",y_true.shape)
        
    #loss = tf.mean(np.square(y_true - y_pred), axis = [1,2,3])
    return loss

/opt/conda/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
batch_count=0
def train_step(orig_images,noisy_images,valid=False):
 
    with tf.GradientTape() as encoder, tf.GradientTape() as decoder:
        latent = enc(noisy_images, training=not(valid))
        generated_images = dec(latent)
        global batch_count
        batch_count+=1
        if(batch_count%50==0):
            rand_index=np.random.randint(0,50)
            print("\t\tNOISY\t\t\t\tDENOISED\t\t\t ORIGINAL")
            fig=plt.figure()
            fig = plt.figure(figsize=(8, 8))
            fig.add_subplot(1,3,1)
            plt.imshow(noisy_images[rand_index])
            fig.add_subplot(1,3,2)
            plt.imshow(generated_images[rand_index])
            fig.add_subplot(1,3,3)
            plt.imshow(orig_images[rand_index])
            plt.show()
            batch_count=0
        
        #Here the Noisy Images were encoded, but the loss is being calculated
        #with respect to the Original Images.
        loss = ae_loss(orig_images, generated_images)
    if(not valid):     
        gradients_of_model = encoder.gradient(loss, enc.trainable_variables)
        gradients_of_dec = decoder.gradient(loss, dec.trainable_variables)


        optimizer.apply_gradients(zip(gradients_of_model,enc.trainable_variables))
        optimizer.apply_gradients(zip(gradients_of_dec, dec.trainable_variables))
    return loss

In [ ]:
def train(orig_images,noised_images, epochs):
  for epoch in range(epochs):
    start = time.time()
    for image_batch,noisy_batch in zip(orig_images,noised_images):
          train_step(image_batch,noisy_batch)
#     for image_batch,noisy_batch in zip(test_dataset,noised_test_dataset):
#         print(f'Valid Loss for Epoch {epoch} is {train_step(image_batch,noisy_batch,valid=True)}')
    if(epoch%50==0):
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
 
train(train_dataset, noised_dataset,200)

In [ ]:
enc.save('/kaggle/working/Encoder.keras')
dec.save("/kaggle/working/Decoder.keras")